# Visualize Networkss

In [2]:
# Mathematical and Data Managment
import numpy as np
import pandas as pd
import scipy.sparse as sp

# Graph Managment
import graph_tool.all as gt
import networkx as nx
from utils.Functions import *
from utils.Segregation import *

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from forceatlas2 import forceatlas2_networkx_layout

# Miscellaneous
from glob import glob
from tqdm import tqdm
import random
import time

/mnt/disk2/anaconda3/envs/gt_global/lib/python3.11/site-packages/graph_tool/draw/cairo_draw.py:1544: RuntimeWarning: Error importing Gtk module: ; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)


In [8]:
# Load the graph
g = gt.load_graph("/mnt/disk2/Data/Daily_graphs/2021-05-03.graphml")
master_id = pd.read_csv('/mnt/disk2/Data/3_Day_Graphs/Nodes.csv', sep = ';')
print(g)
master_id.head()

<Graph object, directed, with 25939 vertices and 262464 edges, 3 internal vertex properties, 1 internal edge property, at 0x7fb33458d4d0>


,Idx in Graph,ID,Label,Political Affiliation,Color
0,0,12,0,No Retweets,"(0, 0, 0, 0.7)"
1,1,12996,alerios,Izquierda,"(0, 0, 1, 0.7)"
2,2,777978,ranaberden,Centro,"(0, 1, 0, 0.7)"
3,3,784125,jsanti,Izquierda,"(0, 0, 1, 0.7)"
4,4,1061601,thisgoblin,Izquierda,"(0, 0, 1, 0.7)"


In [11]:
g.vp['ID'][look_up_graph(g,3)]

KeyError: ('v', 'name')

In [4]:
pos = {i: (random.random(), random.random()) for i in master_id['Idx in Graph']}

# Calculate aesthetics fo visualization
colors = list(master_id['Color'])

In [6]:
ls = []
files = glob('/mnt/disk2/Data/Daily_graphs/*.graphml')

for file in files:
    g = gt.load_graph(file)
    N = g.num_vertices()
    ls.append((file,N))

sorted_ls = sorted(ls, key=lambda x: x[1], reverse=True)
sorted_files = [g[0] for g in sorted_ls]

In [1]:
times = []
for idx, file in tqdm(enumerate(sorted_files)):
    
    start_time = time.time()
    
    g = gt.load_graph(file)
    g_nx = to_networkx(g)
    date = file.split('.')[-2].split('/')[-1]
    
    # Node Size
    node_in_degree = [g_nx.in_degree(node) for node in g_nx.nodes]

    # For Layout Algorythim, calculate it as undirected
    g_nx_undir = g_nx.to_undirected()
    
    # Initalizice Layout
    if idx == 0:

        # Apply ForceAtlas2 layout algorithm First time
        layout = forceatlas2_networkx_layout(g_nx_undir, pos, niter=30)
        print('First Layout')
    
    else:

        # Apply ForceAtlas2 layout algorithm
        layout = forceatlas2_networkx_layout(g_nx_undir, pos, niter=15)
    
    # Re assign new map
    for key in layout:
        if key in pos:
            pos[key] = layout[key]
    
    # Draw the network using matplotlib
    nx.draw_networkx_nodes(g_nx_undir, layout, node_color=colors,
                                    node_size = node_in_degree,
                                    edgecolors=(0,0,0,0.1))

    # Display the plot
    plt.savefig(f'../{date}_FA2_viz.png') 
    
    if idx%10 == 0:
        print(f'Grafo número {idx}')
    else:
        pass
    
    end_time = time.time()
    t = end_time - start_time
    times.append(t)

NameError: name 'tqdm' is not defined